In [1]:
evaluation = False
evaluation_verbose = False

OUTPUT_BUCKET_FOLDER = "./preprocessed/"
DATA_BUCKET_FOLDER = "./dataset/"
#SPARK_TEMP_FOLDER = "/outbrain/spark-temp/"

In [2]:
import numpy as np

import math
import datetime
import time

import pandas as pd

import cudf
print(cudf.__version__)
from numba import cuda

import nvtabular as nvt

from nvtabular.ops import Normalize, FillMissing, Categorify, LogOp, ZeroFill, JoinExternal, Dropna, LambdaOp

0.15.0a+2961.gce826c57c


In [3]:
# def truncate_day_from_timestamp(ts):
#     return int(ts / 1000 / 60 / 60 / 24)

In [4]:
country_utc_dst_cudf = cudf.read_csv('./dataset/data/country_codes_utc_dst_tz_delta.csv', keep_default_na=False)
us_states_utc_dst_cudf = cudf.read_csv('./dataset/data/us_states_abbrev_bst.csv', keep_default_na=False)
ca_states_utc_dst_cudf = cudf.read_csv('./dataset/data/ca_states_abbrev_bst.csv', keep_default_na=False)

In [14]:
us_states_utc_dst_cudf.head()

,state_abb,utc_dst_time_offset_cleaned
0,AL,-5.0
1,AK,-8.0
2,AZ,-7.0
3,AR,-5.0
4,CA,-7.0


In [25]:
us_states_utc_dst_cudf.shape

(51, 2)

In [41]:
country_utc_dst_cudf

,country_code,utc_dst_time_offset_cleaned
0,AX,3.0
1,AF,4.5
2,AL,2.0
3,DZ,1.0
4,AD,2.0
...,...,...
241,VI,-4.0
242,WF,12.0
243,YE,3.0
244,ZM,2.0


In [11]:
country_utc_dst_cudf[country_utc_dst_cudf['country_code'] == 'KP']

,country_code,utc_dst_time_offset_cleaned
110,KP,-4.0
112,KP,8.5
114,KP,9.0


In [15]:
ca_states_utc_dst_cudf.head()

,state_abb,utc_dst_time_offset_cleaned
0,AB,-6.0
1,BC,-7.0
2,MB,-5.0
3,NB,-3.0
4,NL,-3.0


In [21]:
us_state_geo = us_states_utc_dst_cudf.rename(columns ={'state_abb':'geo_code'})
us_state_geo['geo_code'] = 'US>' + us_state_geo['geo_code']
country_geo= country_utc_dst_cudf.rename(columns={'country_code':'geo_code'})
ca_state_geo = ca_states_utc_dst_cudf.rename(columns ={'state_abb':'geo_code'})
ca_state_geo['geo_code'] = 'CA>' + ca_state_geo['geo_code']
geo_code = cudf.concat([us_state_geo,country_geo,ca_state_geo])
geo_code = geo_code.groupby('geo_code').agg('max').reset_index()

In [44]:
geo_code[geo_code['geo_code'] =='US>MD']

,geo_code,utc_dst_time_offset_cleaned
255,US>MD,-4.0


In [5]:
events_cudf = cudf.read_csv(
    DATA_BUCKET_FOLDER + 'events.csv',
    names=['display_id', 'uuid_event', 'document_id_event', 'timestamp_event', 'platform_event', 
           'geo_location_event'], keep_default_na=False, skiprows=1)
# events_cudf['day_event'] = (events_cudf['timestamp_event'] / 1000 / 60 / 60 / 24).astype(int)
# events_cudf['event_country'] = events_cudf['geo_location_event'].str.slice(0,2)
# events_cudf['event_country_state'] = events_cudf['geo_location_event'].str.slice(0,5)

# # Drop rows with empty "geo_location"
# events_cudf = events_cudf.dropna(subset="geo_location_event")
# # Drop rows with empty "platform"
# events_cudf = events_cudf.dropna(subset="platform_event")


In [67]:
events_cudf.head()

,display_id,uuid_event,document_id_event,timestamp_event,platform_event,geo_location_event,day_event,event_country,event_country_state
0,1,cb8c55702adb93,379743,61,3,US>SC>519,0,US,US>SC
1,2,79a85fa78311b9,1794259,81,2,US>CA>807,0,US,US>CA
2,3,822932ce3d8757,1179111,182,2,US>MI>505,0,US,US>MI
3,4,85281d0a49f7ac,1777797,234,2,US>WV>564,0,US,US>WV
4,5,8d0daef4bf5b56,252458,338,2,SG>00,0,SG,SG>00


## NVTabular

In [15]:
proc = nvt.Workflow(
    cat_names=['display_id', 'uuid_event', 'document_id_event', 'timestamp_event',
       'platform_event', 'geo_location_event'],
    cont_names=[],
    label_name=[])


proc.add_preprocess(
    [
    LambdaOp(
        op_name='country',
        f=lambda col, gdf: col.str.slice(0,2),
        columns=['geo_location_event'],
        replace=False
    ),
    LambdaOp(
        op_name='day',
        f=lambda col, gdf: (col / 1000 / 60 / 60 / 24).astype(int),
        columns=['timestamp_event'],
        replace=False
    ),
    LambdaOp(
    op_name='state',
    f=lambda col, gdf: col.str.slice(0,5),
    columns=['geo_location_event'],
    replace=False
    )
    ]
)
#apply dropna
proc.add_feature(
    [
    Dropna(['geo_location_event']),
    Dropna(['platform_event'])
    ])

proc.finalize()

train_dataset = nvt.Dataset(events_cudf, engine='parquet')

proc.apply(train_dataset, apply_offline=True, record_stats=True, output_path='./dataset/test/', shuffle=False, out_files_per_proc=1)

In [18]:
events_cudf= cudf.read_parquet('./dataset/test/*.parquet')
events_cudf= events_cudf.rename(columns={'geo_location_event_country': 'event_country', 'timestamp_event_day':'day_event', 'geo_location_event_state':'event_country_state'})

In [19]:
events_cudf.head()

,display_id,uuid_event,document_id_event,timestamp_event,platform_event,geo_location_event,event_country,day_event,event_country_state
0,36498,690e5a08d98972,1778533,2732246,2,US>AZ>753,US,0,US>AZ
1,19144,d6cfab5bb42fdd,1788332,1354016,2,US>CA>803,US,0,US>CA
2,32963,f1e74a5cf471bb,1782912,2437060,2,US>TN>640,US,0,US>TN
3,14560,edfb091eb20ecb,1760775,1015594,3,US>CA>803,US,0,US>CA
4,40603,233a2b3ca9a56f,1717049,3079057,1,US>UT>770,US,0,US>UT


In [13]:
page_views_cudf = cudf.read_csv(
    DATA_BUCKET_FOLDER + 'page_views.csv',
    names=['uuid_pv', 'document_id_pv', 'timestamp_pv', 'platform_pv', 'geo_location_pv', 'traffic_source_pv'],
    dtype={'uuid_pv': 'str', 'document_id_pv': 'int64', 'timestamp_pv': 'int64',
           'platform_pv': 'int64', 'geo_location_pv': 'str', 'traffic_source_pv': 'int64'},
    skiprows=1,
    keep_default_na=False)

page_views_cudf['day_pv'] = (page_views_cudf['timestamp_pv'] / 1000 / 60 / 60 / 24).astype(int)

page_views_users_cudf = page_views_cudf.groupby(by=['uuid_pv', 'document_id_pv'])['timestamp_pv'].agg('max').reset_index()


In [20]:
promoted_content_cudf = cudf.read_csv(
    DATA_BUCKET_FOLDER + 'promoted_content.csv', 
    names=['ad_id', 'document_id_promo', 'campaign_id', 'advertiser_id'],
    dtype={'ad_id': 'int64','document_id_promo': 'int64', 'campaign_id': 'int64',
        'advertiser_id': 'int64'},
    skiprows=1)

promoted_content_cudf.head(2)

,ad_id,document_id_promo,campaign_id,advertiser_id
0,1,6614,1,7
1,2,471467,2,7


In [22]:
documents_meta_cudf = cudf.read_csv(
    DATA_BUCKET_FOLDER + 'documents_meta.csv',
    names=['document_id_doc', 'source_id', 'publisher_id', 'publish_time'], 
    dtype={'document_id_doc': 'int64', 'source_id':'int64','publisher_id':'int64','publish_time':'date'},
    na_values='\\N',
    skiprows=1)

documents_meta_cudf.head(2)

,document_id_doc,source_id,publisher_id,publish_time
0,1595802,1,603,2016-06-05 00:00:00
1,1524246,1,603,2016-05-26 11:00:00


In [16]:
documents_meta_cudf['publish_time']

0         2016-06-05 00:00:00
1         2016-05-26 11:00:00
2         2016-05-27 00:00:00
3         2016-06-07 00:00:00
4         2016-06-20 00:00:00
                  ...        
2999329                   NaT
2999330                   NaT
2999331                   NaT
2999332                   NaT
2999333                   NaT
Name: publish_time, Length: 2999334, dtype: datetime64[ms]

In [23]:
documents_meta_cudf = documents_meta_cudf[documents_meta_cudf['source_id'].isnull() != True]
print(len(documents_meta_cudf))
source_publishers_cudf = documents_meta_cudf[['source_id','publisher_id']].drop_duplicates()
rows_no_pub_cudf = source_publishers_cudf[source_publishers_cudf['publisher_id'].isnull()]
source_ids_without_publisher_cudf = rows_no_pub_cudf['source_id']
len(source_ids_without_publisher_cudf)

2996816


5058

In [20]:
max_pub_cudf = int(source_publishers_cudf['publisher_id'].max())
max_pub_cudf

1263

In [21]:
new_publishers_cudf = cudf.DataFrame({'source_id':source_ids_without_publisher_cudf,'publisher_id':range(max_pub_cudf+1,max_pub_cudf+1+len(source_ids_without_publisher_cudf) )})

In [22]:
fixed_source_publishers_cudf = cudf.concat([source_publishers_cudf.dropna(),new_publishers_cudf])

In [23]:
fixed_source_publishers_cudf

,source_id,publisher_id
0,1,603
12867,2,85
12874,3,740
29583,4,213
29597,5,184
...,...,...
2990886,14354,6317
2990902,14355,6318
2990906,14356,6319
2991236,14365,6320


In [24]:
documents_meta_cudf = documents_meta_cudf.drop(columns=['publisher_id']).merge(fixed_source_publishers_cudf,how='inner',on='source_id')

In [25]:
events_joined_cudf = events_cudf.merge( \
    documents_meta_cudf.rename(columns={'source_id':'source_id_doc_event','publisher_id':'publisher_doc_event',\
    'publish_time':'publish_time_doc_event'}), \
    left_on='document_id_event',right_on= 'document_id_doc', how='left').merge(page_views_cudf,\
    left_on=['uuid_event','document_id_event','platform_event','geo_location_event','day_event'],\
    right_on=['uuid_pv','document_id_pv','platform_pv','geo_location_pv','day_pv'],how='left')


In [26]:
documents_categories_cudf = cudf.read_csv(
    DATA_BUCKET_FOLDER + 'documents_categories.csv',
    names=['document_id_cat', 'category_id', 'confidence_level_cat'],
    dtype={'document_id_cat': 'int64','category_id': 'int64','confidence_level_cat': 'float32'},
    skiprows=1)
documents_categories_cudf


,document_id_cat,category_id,confidence_level_cat
0,1595802,1611,0.920000
1,1595802,1610,0.070000
2,1524246,1807,0.920000
3,1524246,1608,0.070000
4,1617787,1807,0.920000
...,...,...,...
5481470,777861,1902,0.118806
5481471,599468,1902,0.257000
5481472,599468,1914,0.002000
5481473,774527,1914,0.117633


In [28]:
def top_id_cat_by_confidence(df, top=1,columns=['category_id','confidence_level_cat'],sortby='confidence_level_cat',groupedby='document_id_cat'):
    # Gets a sorted list of (top) rows with largest values in the (sortby) column,
    df_sorted = df.nlargest(top,sortby)
    
    # The row needs to include the value it was grouped on for the final table to be complete
    row = [df_sorted.iloc[0][groupedby]]
    
    # Adding the values from the topN rows as columns. If you want column x, the top 3 values will be labeled x_1,x_2,x_3
    for column in columns:
        row += [df_sorted.iloc[i][column] for i in range(len(df_sorted))] + \
               [None for i in range(top-len(df_sorted))]
        
    
    
    #building the header to reflect that we are creating (top) columns for each column in the input
    header = [groupedby]
    for column in columns:
        header = header +  [column + '_' + str(i+1) for i in range(top)]

    #building the output dataframe
    out_df = cudf.DataFrame([row],columns=header)
    print(out_df)
    
    
    
    
    
        
    
    return out_df
    

In [29]:
def documents_categories_groupby_list(confidence_level_cat, category_id, confidence_level_cat_1, confidence_level_cat_2,category_id_1,category_id_2,group_count_cat):
    if(len(confidence_level_cat) > 1):
        confidence_level_cat_1[0] = confidence_level_cat[0]
        confidence_level_cat_2[0] = confidence_level_cat[1]
        category_id_1[0] = category_id[0]
        category_id_2[0] = category_id[1]
        confidence_level_cat_1[1],confidence_level_cat_2[1] = -1,-1
    else:
        confidence_level_cat_1[0] = confidence_level_cat[0]
        confidence_level_cat_2[0] = 0
        category_id_1[0] = category_id[0]
        
    group_count_cat[0] = len(category_id)

In [30]:
result = documents_categories_cudf.groupby('document_id_cat').apply_grouped(documents_categories_groupby_list,
                              incols=['confidence_level_cat', 'category_id'],
                              outcols={'confidence_level_cat_1': np.float32,
                                       'confidence_level_cat_2': np.float32,
                                       'category_id_1':np.int32,
                                       'category_id_2':np.int32,
                                       'group_count_cat':np.int32},
                              # threads per block
                              tpb=1)
document_categories_grouped_cudf = result[result['group_count_cat'] != 0]

In [31]:
def documents_topics_groupby_list(topic_id,confidence_level_top,topic_id_1,topic_id_2,topic_id_3, confidence_level_top_1,confidence_level_top_2,confidence_level_top_3,group_count_top):
    group_len = len(confidence_level_top)
    if group_len > 0:
        topic_id_1[0] = topic_id[0]
        confidence_level_top_1[0] = confidence_level_top[0]
    if group_len > 1:
        topic_id_2[0] = topic_id[1]
        confidence_level_top_2[0] = confidence_level_top[1]
    if group_len > 2:
        topic_id_3[0] = topic_id[2]
        confidence_level_top_3[0] = confidence_level_top[2]
    group_count_top[0] = group_len
        
        
        

In [32]:
documents_topics_cudf = cudf.read_csv(
    DATA_BUCKET_FOLDER + 'documents_topics.csv',
    names=['document_id_top', 'topic_id', 'confidence_level_top'],
    dtype={'document_id_top': 'int64','topic_id': 'int64','confidence_level_top': 'float32'},
    skiprows=1)
documents_topics_cudf

,document_id_top,topic_id,confidence_level_top
0,1595802,140,0.073113
1,1595802,16,0.059416
2,1595802,143,0.045421
3,1595802,170,0.038867
4,1524246,113,0.196450
...,...,...,...
11325955,1080024,221,0.010094
11325956,1125054,29,0.040381
11325957,1080122,29,0.038200
11325958,1291726,68,0.069515


In [33]:
result = documents_topics_cudf.groupby('document_id_top').apply_grouped(documents_topics_groupby_list,
                              incols=['confidence_level_top', 'topic_id'],
                              outcols={'confidence_level_top_1': np.float32,
                                       'confidence_level_top_2': np.float32,
                                       'confidence_level_top_3': np.float32,
                                       'topic_id_1':np.int32,
                                       'topic_id_2':np.int32,
                                       'topic_id_3':np.int32,
                                       'group_count_top':np.int32},
                              # threads per block
                              tpb=1)
result[result['group_count_top']!=0]

,document_id_top,topic_id,confidence_level_top,confidence_level_top_1,confidence_level_top_2,confidence_level_top_3,group_count_top,topic_id_1,topic_id_2,topic_id_3
4504005,1,252,0.020726,0.020726,0.012773,0.011959,5,252,35,150
937048,2,216,0.013029,0.013029,0.000000,0.000000,1,216,0,0
164161,3,276,0.029914,0.029914,0.027589,0.019660,13,276,107,176
4996667,4,102,0.084171,0.084171,0.083300,0.077338,6,102,75,16
4996673,5,75,0.106555,0.106555,0.086467,0.054225,9,75,102,244
...,...,...,...,...,...,...,...,...,...,...
3292666,2999325,285,0.028784,0.028784,0.000000,0.000000,1,285,0,0
3292667,2999327,271,0.038198,0.038198,0.000000,0.000000,1,271,0,0
3292668,2999330,9,0.035086,0.035086,0.000000,0.000000,1,9,0,0
3292669,2999332,185,0.012333,0.012333,0.000000,0.000000,1,185,0,0


In [34]:
documents_entities_cudf = cudf.read_csv(
    DATA_BUCKET_FOLDER + 'documents_entities.csv',

    names=['document_id_ent', 'entity_id', 'confidence_level_ent'],
    dtype={'document_id_ent': 'int64','entity_id': 'str','confidence_level_ent': 'float32'},
    skiprows=1)


In [35]:
def documents_entities_groupby_list(entity_id,confidence_level_ent, entity_id_1,entity_id_2,entity_id_3,entity_id_4,entity_id_5,entity_id_6,confidence_level_ent_1,confidence_level_ent_2,confidence_level_ent_3,confidence_level_ent_4,confidence_level_ent_5,confidence_level_ent_6,group_count_ent):
    group_len = len(confidence_level_top)
    if group_len > 0:
        entity_id_1[0] = entity_id[0]
        confidence_level_ent_1[0] = confidence_level_ent[0]
    if group_len > 1:
        entity_id_2[0] = entity_id[1]
        confidence_level_ent_2[0] = confidence_level_ent[1]
    if group_len > 2:
        entity_id_3[0] = entity_id[2]
        confidence_level_ent_3[0] = confidence_level_ent[2]
    if group_len > 3:
        entity_id_4[0] = entity_id[3]
        confidence_level_ent_4[0] = confidence_level_ent[3]
    if group_len > 4:
        entity_id_5[0] = entity_id[4]
        confidence_level_ent_5[0] = confidence_level_ent[4]
    if group_len > 5:
        entity_id_6[0] = entity_id[5]
        confidence_level_ent_6[0] = confidence_level_ent[5]
    group_count_ent[0] = group_len

In [36]:
# result = documents_entities_cudf.groupby('document_id_ent').apply_grouped(documents_entities_groupby_list,
#                               incols=['confidence_level_ent', 'entity_id'],
#                               outcols={'confidence_level_ent_1': np.float32,
#                                        'confidence_level_ent_2': np.float32,
#                                        'confidence_level_ent_3': np.float32,
#                                        'confidence_level_ent_4': np.float32,
#                                        'confidence_level_ent_5': np.float32,
#                                        'confidence_level_ent_6': np.float32,
#                                        'topic_id_1':np.int32,
#                                        'topic_id_2':np.int32,
#                                        'topic_id_3':np.int32,
#                                        'topic_id_4':np.int32,
#                                        'topic_id_5':np.int32,
#                                        'topic_id_6':np.int32,
#                                        'group_count_ent':np.int32},
#                               # threads per block
#                               tpb=1)
print("ERRORS")

ERRORS


In [37]:
clicks_train_cudf = cudf.read_csv(
    DATA_BUCKET_FOLDER + 'clicks_train.csv',
    names=['display_id', 'ad_id', 'clicked'],
    dtype={'display_id': 'int64','ad_id': 'int64','clicked': 'int64'},
    skiprows=1)


In [38]:
clicks_train_cudf.merge(promoted_content_cudf,left_on='ad_id',right_on='ad_id',how='left')\
    .merge(documents_meta_cudf,left_on='document_id_promo',right_on='document_id_doc',how = 'left')

,display_id,ad_id,clicked,document_id_promo,campaign_id,advertiser_id,document_id_doc,source_id,publish_time,publisher_id
0,4633,155551,1,1159254,13669,2028,1159254,8834,NaT,3514
1,4633,184717,0,1462705,19291,490,1462705,4976,2015-11-11,621
2,4633,252931,0,916726,26324,2033,916726,8404,2015-10-22,3337
3,4633,273936,0,1472170,16425,3042,1472170,11774,NaT,4443
4,4634,42120,1,928443,5845,1438,928443,7437,2015-11-02,920
...,...,...,...,...,...,...,...,...,...,...
99994,18113,107383,1,463357,12332,47,463357,1137,NaT,1452
99995,18113,120236,0,356726,15143,579,356726,4394,2013-10-29,503
99996,18113,147704,0,954430,10363,2603,954430,8662,2016-07-16,3461
99997,18113,233298,0,953052,6277,670,953052,5752,2016-07-08,2141


In [39]:
clicks_train_joined_cudf = clicks_train_cudf.merge(promoted_content_cudf,left_on='ad_id',right_on='ad_id',how='left')\
    .merge(documents_meta_cudf,left_on='document_id_promo',right_on='document_id_doc',how = 'left')\
    .merge(events_joined_cudf, left_on='display_id',right_on= 'display_id',how='left')
clicks_train_joined_cudf.rename(columns={'document_id_doc_x':'document_id_doc'},inplace=True)

In [40]:
clicks_train_joined_cudf

,display_id,ad_id,clicked,document_id_promo,campaign_id,advertiser_id,document_id_doc,source_id,publish_time,publisher_id,...,source_id_doc_event,publish_time_doc_event,publisher_doc_event,uuid_pv,document_id_pv,timestamp_pv,platform_pv,geo_location_pv,traffic_source_pv,day_pv
0,7865,89858,0,1090800,11635,2727,1090800,8533,2016-02-11 21:00:00,3409,...,6872,2016-06-12 00:00:00,41,None,null,null,null,None,null,null
1,7865,137128,0,1225364,17621,2988,1225364,11604,NaT,4384,...,6872,2016-06-12 00:00:00,41,None,null,null,null,None,null,null
2,7865,149047,0,1169985,16636,380,1169985,2826,NaT,440,...,6872,2016-06-12 00:00:00,41,None,null,null,null,None,null,null
3,7865,154256,0,1179134,19654,3251,1179134,6882,2016-03-14 08:00:00,1143,...,6872,2016-06-12 00:00:00,41,None,null,null,null,None,null,null
4,7865,201660,0,1513530,11164,2691,1513530,4859,2016-05-25 10:00:00,1886,...,6872,2016-06-12 00:00:00,41,None,null,null,null,None,null,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,16392,144740,0,1337362,18488,2909,1337362,11328,2016-04-24 08:00:00,4312,...,3,2016-07-13 00:00:00,740,None,null,null,null,None,null,null
99995,16392,148140,0,478325,16554,102,478325,3199,2015-01-26 00:00:00,341,...,3,2016-07-13 00:00:00,740,None,null,null,null,None,null,null
99996,16392,151028,0,1270285,18075,865,1270285,766,2014-02-24 00:00:00,495,...,3,2016-07-13 00:00:00,740,None,null,null,null,None,null,null
99997,16393,5124,0,21139,120,24,21139,1871,2016-05-31 00:00:00,156,...,10334,2016-06-13 07:00:00,440,None,null,null,null,None,null,null


In [41]:
def get_category_field_values_counts(field, df, min_threshold=10):
    category_counts = dict(list(filter(lambda x: not is_null(x[0]) and x[1] >= min_threshold, df.select(field).groupBy(field).count().rdd.map(lambda x: (x[0], x[1])).collect())))
    #Adding a special value to create a feature for values in this category that are less than min_threshold 
    category_counts[LESS_SPECIAL_CAT_VALUE] = -1
    return category_counts


In [42]:
LESS_SPECIAL_CAT_VALUE = 'less'

In [43]:
def is_null(value):
    return value is None or len(str(value).strip()) == 0


In [44]:
 def get_category_field_values_counts_cudf(field, df, min_threshold=10):
    df = df.groupby(field)[field].agg('count').to_frame()


    df.rename({field:'count'},inplace =True)

    df = df[df['count'] >= min_threshold].reset_index()
    df = cudf.concat([df,cudf.DataFrame({"event_country":['less'], "count":[-1]})],ignore_index=True)
    return df

In [45]:
dropna = get_category_field_values_counts_cudf('event_country', events_cudf, min_threshold=10)
#All non-null categories: 230

event_country_state_values_counts = get_category_field_values_counts_cudf('event_country_state', events_cudf, min_threshold=10)

event_geo_location_values_counts = get_category_field_values_counts_cudf('geo_location_event', events_cudf, min_threshold=10)
#All non-null categories: 2988

doc_entity_id_values_counts = get_category_field_values_counts_cudf('entity_id', documents_entities_cudf, min_threshold=10)


In [46]:
documents_total = len(documents_meta_cudf)
documents_total


2996816

In [47]:
def get_ctr(df):
    return df['clicks'] /df['views']

In [48]:
def gather_stats(gdf):
    avg_ctr = gdf['ctr'].sum() / len(gdf['ctr'])
    weighted_avg_ctr = (gdf['ctr'] * gdf['views']).sum() / gdf['views'].sum()
    median_views = gdf['views'].median()
    mean_views= gdf['views'].mean()
    return avg_ctr,weighted_avg_ctr,median_views,mean_views


In [49]:
def get_popularity_cudf(gdf, fields,get_distinct_ad_ids = True,additional_output=[]):
    grouped = gdf.groupby(fields)
    gdf = grouped['clicked'].agg(['sum','count']).rename({'sum':'clicks','count':'views'})
    if(get_distinct_ad_ids):
        gdf['distinct_ad_ids'] = grouped['ad_id'].nunique()
    gdf['ctr'] = get_ctr(gdf)
    for field in additional_output:
        gdf[field[0]] = grouped[field[1]].agg([field[2]])
    gdf.reset_index(inplace=True)
    final_fields = fields + ['ctr','views']
    if(get_distinct_ad_ids):
        final_fields.append('distinct_ad_ids')
    final_fields += [field[0] for field in additional_output]
    gdf = gdf[final_fields]
   
    return gdf

In [50]:
train_set_cudf = clicks_train_joined_cudf

In [51]:
train_set_cudf

,display_id,ad_id,clicked,document_id_promo,campaign_id,advertiser_id,document_id_doc,source_id,publish_time,publisher_id,...,source_id_doc_event,publish_time_doc_event,publisher_doc_event,uuid_pv,document_id_pv,timestamp_pv,platform_pv,geo_location_pv,traffic_source_pv,day_pv
0,7865,89858,0,1090800,11635,2727,1090800,8533,2016-02-11 21:00:00,3409,...,6872,2016-06-12 00:00:00,41,None,null,null,null,None,null,null
1,7865,137128,0,1225364,17621,2988,1225364,11604,NaT,4384,...,6872,2016-06-12 00:00:00,41,None,null,null,null,None,null,null
2,7865,149047,0,1169985,16636,380,1169985,2826,NaT,440,...,6872,2016-06-12 00:00:00,41,None,null,null,null,None,null,null
3,7865,154256,0,1179134,19654,3251,1179134,6882,2016-03-14 08:00:00,1143,...,6872,2016-06-12 00:00:00,41,None,null,null,null,None,null,null
4,7865,201660,0,1513530,11164,2691,1513530,4859,2016-05-25 10:00:00,1886,...,6872,2016-06-12 00:00:00,41,None,null,null,null,None,null,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,16392,144740,0,1337362,18488,2909,1337362,11328,2016-04-24 08:00:00,4312,...,3,2016-07-13 00:00:00,740,None,null,null,null,None,null,null
99995,16392,148140,0,478325,16554,102,478325,3199,2015-01-26 00:00:00,341,...,3,2016-07-13 00:00:00,740,None,null,null,null,None,null,null
99996,16392,151028,0,1270285,18075,865,1270285,766,2014-02-24 00:00:00,495,...,3,2016-07-13 00:00:00,740,None,null,null,null,None,null,null
99997,16393,5124,0,21139,120,24,21139,1871,2016-05-31 00:00:00,156,...,10334,2016-06-13 07:00:00,440,None,null,null,null,None,null,null


In [145]:

ad_id_popularity_cudf = get_popularity_cudf(train_set_cudf,['ad_id'],False)
ad_id_popularity_cudf = ad_id_popularity_cudf[ad_id_popularity_cudf['views'] >5]
ad_id_avg_ctr_cudf,ad_id_weighted_avg_ctr_cudf,ad_id_views_median_cudf,ad_id_views_mean_cudf= gather_stats(ad_id_popularity_cudf)
ad_id_popularity_cudf = ad_id_popularity_cudf
ad_id_popularity_cudf['distinct_ad_ids'] = 1
ad_id_popularity_cudf['confidence'] = get_confidence_sample_size( \
            ad_id_popularity_cudf['views'] / \
            ad_id_popularity_cudf['distinct_ad_ids'].astype(float))
ad_id_popularity_cudf

,ad_id,ctr,views,distinct_ad_ids,confidence
40,2220,0.000000,6,1,0.117155
44,2316,0.000000,8,1,0.132286
50,2445,0.500000,8,1,0.132286
70,3010,0.666667,6,1,0.117155
73,3021,0.285714,7,1,0.125195
...,...,...,...,...,...
15721,347571,0.285714,14,1,0.163041
15726,347830,0.058824,17,1,0.174018
15731,347979,0.000000,6,1,0.117155
15734,348054,0.142857,7,1,0.125195


In [146]:

document_id_popularity_cudf = get_popularity_cudf(train_set_cudf,['document_id_promo'])
document_id_popularity_cudf = document_id_popularity_cudf[document_id_popularity_cudf['views'] >5]
document_id_avg_ctr_cudf,document_id_weighted_avg_ctr_cudf,document_id_views_median_cudf,document_id_views_mean_cudf = gather_stats(document_id_popularity_cudf)
document_id_popularity_cudf = document_id_popularity_cudf
document_id_popularity_cudf['confidence'] = get_confidence_sample_size( \
            document_id_popularity_cudf['views'] / \
            document_id_popularity_cudf['distinct_ad_ids'].astype(float))
document_id_popularity_cudf

,document_id_promo,ctr,views,distinct_ad_ids,confidence
4,12668,0.000000,24,2,0.154425
6,21135,0.222222,27,2,0.161000
7,21139,0.157895,38,2,0.180361
8,21197,0.000000,16,2,0.132286
9,22351,0.090909,22,6,0.092744
...,...,...,...,...,...
8111,1786069,0.117647,85,1,0.268178
8117,1787510,0.058824,17,1,0.174018
8121,1787907,0.142857,7,1,0.125195
8124,1788392,0.250000,24,4,0.117155


In [54]:

doc_event_doc_ad_avg_ctr_cudf = get_popularity_cudf(train_set_cudf,['document_id_event','document_id_promo'])
doc_event_doc_ad_avg_ctr_cudf = doc_event_doc_ad_avg_ctr_cudf[doc_event_doc_ad_avg_ctr_cudf['views'] >5]
doc_event_doc_ad_avg_ctr_cudf =doc_event_doc_ad_avg_ctr_cudf.set_index(['document_id_event','document_id_promo'])
doc_event_doc_ad_avg_ctr_cudf

ctr  views  distinct_ad_ids
document_id_event document_id_promo                                  
2861              874764             0.285714      7                1
                  1142516            0.833333      6                1
7054              925820             0.285714      7                2
                  942989             0.000000      6                2
                  976846             0.071429     14                6
...                                       ...    ...              ...
1795002           1487909            0.833333      6                1
1795295           1130922            0.111111      9                5
                  1377696            0.000000      6                1
                  1563558            0.111111      9                2
                  1618502            0.000000      6                4

[1057 rows x 3 columns]

In [55]:

source_id_by_country_popularity_cudf = get_popularity_cudf(train_set_cudf,['event_country','source_id'])
source_id_by_country_popularity_cudf = source_id_by_country_popularity_cudf[source_id_by_country_popularity_cudf['views'] >5]
source_id_by_country_avg_ctr_cudf, source_id_by_country_weighted_avg_ctr_cudf,source_id_by_country_views_median_cudf,source_id_by_country_views_mean_cudf = gather_stats(source_id_by_country_popularity_cudf)


In [148]:
source_id_popularity_cudf = get_popularity_cudf(train_set_cudf,['source_id'])
source_id_popularity_cudf = source_id_popularity_cudf[source_id_popularity_cudf['views'] > 10]
source_id_popularity_cudf = source_id_popularity_cudf[source_id_popularity_cudf['source_id'].isnull() != True]
source_id_popularity_cudf['confidence'] = get_confidence_sample_size( \
            source_id_popularity_cudf['views'] / \
            source_id_popularity_cudf['distinct_ad_ids'].astype(float))

In [172]:
publisher_popularity_cudf = get_popularity_cudf(train_set_cudf,['publisher_id'])
publisher_popularity_cudf = publisher_popularity_cudf[publisher_popularity_cudf['views'] > 10]
publisher_popularity_cudf = publisher_popularity_cudf[publisher_popularity_cudf['publisher_id'].isnull() != True]
publisher_popularity_cudf['confidence'] = get_confidence_sample_size( \
            publisher_popularity_cudf['views'] / \
            publisher_popularity_cudf['distinct_ad_ids'].astype(float))

In [175]:
advertiser_id_popularity_cudf = get_popularity_cudf(train_set_cudf,['advertiser_id'])
advertiser_id_popularity_cudf = advertiser_id_popularity_cudf[advertiser_id_popularity_cudf['views'] > 10]
advertiser_id_popularity_cudf = advertiser_id_popularity_cudf[advertiser_id_popularity_cudf['advertiser_id'].isnull() != True]
advertiser_id_popularity_cudf['confidence'] = get_confidence_sample_size( \
            advertiser_id_popularity_cudf['views'] / \
            advertiser_id_popularity_cudf['distinct_ad_ids'].astype(float))

In [174]:
campaign_id_popularity_cudf = get_popularity_cudf(train_set_cudf,['campaign_id'])
campaign_id_popularity_cudf = campaign_id_popularity_cudf[campaign_id_popularity_cudf['views'] > 10]
campaign_id_popularity_cudf = campaign_id_popularity_cudf[campaign_id_popularity_cudf['campaign_id'].isnull() != True]
campaign_id_popularity_cudf['confidence'] = get_confidence_sample_size( \
            campaign_id_popularity_cudf['views'] / \
            campaign_id_popularity_cudf['distinct_ad_ids'].astype(float))

In [189]:
train_aug_cat=train_set_cudf.merge(documents_categories_cudf,left_on='document_id_promo',right_on='document_id_cat',how='inner')

category_id_popularity_cudf = get_popularity_cudf(train_aug_cat,['category_id'],additional_output=[['avg_confidence_level_cat','confidence_level_cat','mean']])

category_id_popularity_cudf = category_id_popularity_cudf[category_id_popularity_cudf['views'] > 10]
category_id_popularity_cudf = category_id_popularity_cudf[category_id_popularity_cudf['category_id'].isnull() != True]
category_id_popularity_cudf['confidence'] = get_confidence_sample_size( \
            category_id_popularity_cudf['views'] / \
            category_id_popularity_cudf['distinct_ad_ids'].astype(float)) * \
            category_id_popularity_cudf['avg_confidence_level_cat']
category_id_popularity_cudf

,category_id,ctr,views,distinct_ad_ids,avg_confidence_level_cat,confidence
0,1000,0.251572,159,30,0.382531,0.042389
1,1100,0.200608,4606,843,0.373629,0.041980
2,1200,0.219512,41,2,0.500000,0.092358
3,1202,0.131661,638,116,0.835725,0.094181
4,1203,0.139665,1432,217,0.282237,0.034461
...,...,...,...,...,...,...
87,2003,0.114852,4336,543,0.573052,0.075750
88,2004,0.188953,3440,729,0.412206,0.043275
89,2005,0.072239,2644,285,0.068420,0.009598
90,2006,0.224385,976,164,0.453325,0.052919


In [61]:
category_id_by_country_popularity_cudf = get_popularity_cudf(train_aug_cat,['event_country','category_id'],additional_output=[['avg_confidence_level_cat','confidence_level_cat','mean']])
category_id_by_country_popularity_cudf = category_id_by_country_popularity_cudf[category_id_by_country_popularity_cudf['views'] > 10]
category_id_by_country_popularity_cudf = category_id_by_country_popularity_cudf[category_id_by_country_popularity_cudf['category_id'].isnull() != True]
category_id_by_country_popularity_cudf = category_id_by_country_popularity_cudf.set_index(['event_country','category_id'])

In [188]:
train_aug_top = train_set_cudf.merge(documents_topics_cudf,left_on='document_id_promo',right_on='document_id_top',how='inner')
topic_id_popularity_cudf = get_popularity_cudf(train_aug_top,['topic_id'],additional_output=[['avg_confidence_level_top','confidence_level_top','mean']])
topic_id_popularity_cudf = topic_id_popularity_cudf[topic_id_popularity_cudf['views'] > 10]
topic_id_popularity_cudf['confidence'] = get_confidence_sample_size( \
            topic_id_popularity_cudf['views'] / \
            topic_id_popularity_cudf['distinct_ad_ids'].astype(float)) * \
            topic_id_popularity_cudf['avg_confidence_level_top']
topic_id_popularity_cudf


,topic_id,ctr,views,distinct_ad_ids,avg_confidence_level_top,confidence
0,0,0.248837,860,158,0.140021,0.015705
1,1,0.230483,807,119,0.083858,0.010359
2,2,0.139636,1153,204,0.121395,0.013849
3,3,0.366864,169,55,0.087965,0.007437
4,4,0.381579,380,50,0.074673,0.009674
...,...,...,...,...,...,...
295,295,0.147541,183,35,0.122291,0.013467
296,296,0.210127,3950,457,0.195877,0.026726
297,297,0.255441,873,232,0.043196,0.004059
298,298,0.152000,125,34,0.118777,0.011031


In [63]:

topic_id_by_country_popularity_cudf  = get_popularity_cudf(train_aug_top,['event_country','topic_id'],additional_output=[['avg_confidence_level_top','confidence_level_top','mean']])
topic_id_by_country_popularity_cudf  = topic_id_by_country_popularity_cudf[topic_id_by_country_popularity_cudf['views'] > 10]
topic_id_by_country_popularity_cudf = topic_id_by_country_popularity_cudf[topic_id_by_country_popularity_cudf['event_country'].isnull() != True]
topic_id_by_country_popularity_cudf = topic_id_by_country_popularity_cudf.set_index(['event_country','topic_id'])

In [187]:
train_aug_ent = train_set_cudf.merge(documents_entities_cudf,left_on='document_id_promo',right_on='document_id_ent',how='inner')

entity_id_popularity_cudf = get_popularity_cudf(train_aug_ent,['entity_id'],additional_output=[['avg_confidence_level_ent','confidence_level_ent','mean']])
entity_id_popularity_cudf  = entity_id_popularity_cudf[entity_id_popularity_cudf['views'] > 5]

entity_id_popularity_cudf['confidence'] = get_confidence_sample_size( \
            entity_id_popularity_cudf['views'] / \
            entity_id_popularity_cudf['distinct_ad_ids'].astype(float)) * \
            entity_id_popularity_cudf['avg_confidence_level_ent']
entity_id_popularity_cudf



,entity_id,ctr,views,distinct_ad_ids,avg_confidence_level_ent,confidence
2,000a0ac94dbe028dc102deacf7c90438,0.192308,26,10,0.310450,0.023942
3,000a5439be001a902cb02cce5d36f9a9,0.230769,13,1,0.374884,0.059564
4,000a650637917f62e36381987f1ac69b,0.179798,495,203,0.212446,0.015796
10,003563522ba61cf2bc40324892e8b3f0,0.166667,6,2,0.207532,0.017321
13,004a31e46399eff0a420086406914d0e,0.561321,212,11,0.208850,0.037839
...,...,...,...,...,...,...
13355,ff83e7ab168e9301dcf09cd5d4be2886,0.166667,6,4,0.317936,0.017539
13366,ffb5af9103d80de5bc0a3b4a7c2415c7,0.000000,6,2,0.245446,0.020486
13372,ffdcc9cb813808336e438e805f281821,0.000000,7,1,0.255460,0.031982
13373,ffeb12c0c7a489858e284090db72a128,0.000000,6,4,0.583842,0.032208


In [65]:
entity_id_by_country_popularity_cudf = get_popularity_cudf(train_aug_ent,['event_country','entity_id'],additional_output=[['avg_confidence_level_ent','confidence_level_ent','mean']])
entity_id_by_country_popularity_cudf  = entity_id_by_country_popularity_cudf[entity_id_by_country_popularity_cudf['views'] > 5]
entity_id_by_country_popularity_cudf = entity_id_by_country_popularity_cudf[entity_id_by_country_popularity_cudf['event_country'].isnull() != True]
entity_id_by_country_popularity_cudf = entity_id_by_country_popularity_cudf.set_index(['event_country','entity_id'])

In [66]:
entity_id_by_country_popularity_cudf

ctr  views  \
event_country entity_id                                           
AE            07c145e2973a82ad49230a084b2448a7  0.545455     11   
              0d97f44a882aacb8722350ba12bceb49  0.777778      9   
              13324fb61207ae36ae907f9311c08278  0.055556     18   
              1f1a43da7780b5aaa6102d56061f2244  0.545455     11   
              225d336d0d0ad113024eb5158472a535  0.545455     11   
...                                                  ...    ...   
ZA            f4cfd530a4ab914afd1b19002c12681c  0.000000     12   
              f5a85fd5bfc731c099b3558f3d4d990d  0.625000      8   
              f67cb9001a55554dce40e1b785222770  0.400000     20   
              fb3543252bb0cbe9a8530822c9b70dbb  0.441176     34   
              fec5df8be19e0853258c6b6ba91dd93d  0.000000     12   

                                                distinct_ad_ids  \
event_country entity_id                                           
AE            07c145e2973a82ad49230a084b2448a7                2   
              0d97f44a882aacb8722350ba12bceb49                2   
              13324fb61207ae36ae907f9311c08278                4   
              1f1a43da7780b5aaa6102d56061f2244                2   
              225d336d0d0ad113024eb5158472a535                2   
...                                                         ...   
ZA            f4cfd530a4ab914afd1b19002c12681c                5   
              f5a85fd5bfc731c099b3558f3d4d990d                1   
              f67cb9001a55554dce40e1b785222770                2   
              fb3543252bb0cbe9a8530822c9b70dbb                3   
              fec5df8be19e0853258c6b6ba91dd93d                5   

                                                avg_confidence_level_ent  
event_country entity_id                                                   
AE            07c145e2973a82ad49230a084b2448a7                  0.382079  
              0d97f44a882aacb8722350ba12bceb49                  0.743495  
              13324fb61207ae36ae907f9311c08278                  0.147523  
              1f1a43da7780b5aaa6102d56061f2244                  0.216913  
              225d336d0d0ad113024eb5158472a535                  0.375496  
...                                                                  ...  
ZA            f4cfd530a4ab914afd1b19002c12681c                  0.205654  
              f5a85fd5bfc731c099b3558f3d4d990d                  0.212001  
              f67cb9001a55554dce40e1b785222770                  0.258651  
              fb3543252bb0cbe9a8530822c9b70dbb                  0.298869  
              fec5df8be19e0853258c6b6ba91dd93d                  0.282591  

[5796 rows x 4 columns]

In [67]:
categories_docs_counts_cudf = documents_categories_cudf.groupby('category_id').size()
topics_docs_counts_cudf = documents_topics_cudf.groupby('topic_id').size()
entities_docs_counts_cudf = documents_entities_cudf.groupby('entity_id').size()

In [68]:
categories_docs_counts_cudf

category_id
1000      5074
1100    212249
1200         7
1202      3259
1203     30511
         ...  
2003     53704
2004    159334
2005      4998
2006     27232
2100    125219
Length: 97, dtype: int32

In [69]:
def get_percentiles(cudf, field, quantiles_levels=None, max_error_rate=0.0):
    if quantiles_levels == None:
        quantiles_levels = np.arange(0.0, 1.1, 0.1).tolist() 
    print(quantiles_levels)
    quantiles = cudf[field].quantile(quantiles_levels[0])
    return quantiles


In [70]:
publish_times_cudf = train_set_cudf[train_set_cudf['publish_time'].isnull()!=True]

publish_times_cudf = publish_times_cudf[['document_id_promo','publish_time']].drop_duplicates()
publish_times_cudf['publish_time']= (cudf.to_datetime(publish_times_cudf['publish_time']).astype(int) /10**9).astype(int)


In [71]:
publish_time_median_cudf = publish_times_cudf['publish_time'].median()

In [72]:
publish_time_median_cudf

1458.0

In [73]:
train_set_cudf[train_set_cudf['uuid_pv'].isnull()!=True]

,display_id,ad_id,clicked,document_id_promo,campaign_id,advertiser_id,document_id_doc,source_id,publish_time,publisher_id,...,source_id_doc_event,publish_time_doc_event,publisher_doc_event,uuid_pv,document_id_pv,timestamp_pv,platform_pv,geo_location_pv,traffic_source_pv,day_pv
10839,4186,86683,0,1106570,11201,2670,1106570,10821,2016-02-21,4086,...,10517,2011-08-04 10:00:00,240,fbaee99663e3c9,25792,287493,1,US>OK>650,1,0
10840,4186,97535,1,1140373,2424,1256,1140373,3549,NaT,1637,...,10517,2011-08-04 10:00:00,240,fbaee99663e3c9,25792,287493,1,US>OK>650,1,0
10841,4186,138352,0,786803,17762,1703,786803,8031,2016-07-29,3184,...,10517,2011-08-04 10:00:00,240,fbaee99663e3c9,25792,287493,1,US>OK>650,1,0
13397,3059,82871,0,419768,10146,2585,419768,6286,2014-09-20,819,...,10517,2011-08-04 10:00:00,240,d309dd25386f94,25792,211358,1,NZ>E7,1,0
13398,3059,85830,1,1103090,6886,892,1103090,2613,NaT,62,...,10517,2011-08-04 10:00:00,240,d309dd25386f94,25792,211358,1,NZ>E7,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99761,18855,155250,0,1366277,19302,3190,1366277,12176,2016-04-29,4657,...,10517,2011-08-04 10:00:00,240,d4b482fd2594b2,25792,936987,3,US>CO>752,1,0
99762,18855,167366,0,1019847,20136,446,1019847,9839,2015-12-11,3770,...,10517,2011-08-04 10:00:00,240,d4b482fd2594b2,25792,936987,3,US>CO>752,1,0
99763,18855,173005,0,1031141,21149,1703,1031141,8031,2016-07-29,3184,...,10517,2011-08-04 10:00:00,240,d4b482fd2594b2,25792,936987,3,US>CO>752,1,0
99764,18855,257524,0,1556282,26491,3190,1556282,12176,2016-05-29,4657,...,10517,2011-08-04 10:00:00,240,d4b482fd2594b2,25792,936987,3,US>CO>752,1,0


In [74]:
def get_days_diff(newer_timestamp, older_timestamp):
    sec_diff = newer_timestamp - older_timestamp
    days_diff = sec_diff / 60 / 60 / 24
    return days_diff


In [75]:

TIME_DECAY_ALPHA = 0.0005

ref_dates = [
                1476714880, # 7 days
                1474727680, # 30 days
                1469370880, # 90 days
                1461508480,  # 180 days
                1445697280, # 1 year
                1414161280 # 2 years
]


# ### Get local time


DEFAULT_TZ_EST = -4.0


def get_local_utc_bst_tz(event_country, event_country_state):
    local_tz = DEFAULT_TZ_EST
    if len(event_country) > 0:
        if event_country in countries_utc_dst_dict:
            local_tz = countries_utc_dst_dict[event_country]
            if len(event_country_state)>2:
                state = event_country_state[3:5]
                if event_country == 'US':  
                    if state in us_states_utc_dst_dict:
                        local_tz = us_states_utc_dst_dict[state]                
                elif event_country == 'CA':
                    if state in ca_states_utc_dst_dict:
                        local_tz = ca_states_dst_dict[state] 
    return float(local_tz)


In [76]:
countries_utc_dst_dict =country_utc_dst_cudf.set_index('country_code')['utc_dst_time_offset_cleaned']
us_states_utc_dst_dict =us_states_utc_dst_cudf.set_index('state_abb')['utc_dst_time_offset_cleaned']
ca_states_utc_dst_dict =ca_states_utc_dst_cudf.set_index('state_abb')['utc_dst_time_offset_cleaned']

In [77]:
hour_bins_dict = {'EARLY_MORNING': 0,
             'MORNING': 1,
             'MIDDAY': 2,
             'AFTERNOON': 3,
             'EVENING': 4,
             'NIGHT': 5}

hour_bins_values = sorted(hour_bins_dict.values())

def get_hour_bin(hour):
    if hour >= 5 and hour < 8:
        hour_bin = 0
    elif hour >= 8 and hour < 11:
        hour_bin = 1
    elif hour >= 11 and hour < 14:
        hour_bin = 2
    elif hour >= 14 and hour < 19:
        hour_bin = 3
    elif hour >= 19 and hour < 22:
        hour_bin = 4
    else:
        hour_bin = 5
    return hour_bin

def get_local_datetime(dt, event_country, event_country_state):
    local_tz = get_local_utc_bst_tz(event_country, event_country_state)  
    tz_delta = local_tz - DEFAULT_TZ_EST
    local_time = dt +  datetime.timedelta(hours=tz_delta)
    return local_time

get_local_datetime(datetime.datetime.now(), 'US', 'US>CA')

def is_weekend(dt):
    return dt.weekday() >= 5

is_weekend(datetime.datetime(2016, 6, 14))



False

In [131]:
def date_time_to_unix_epoch(date_time):
  return int(time.mktime(date_time.timetuple()))
def get_time_decay_factor(timestamp, timestamp_ref=None, alpha=0.001):
    if timestamp_ref is None:
        timestamp_ref = time.time()
        
    days_diff = get_days_diff(timestamp_ref, timestamp)
    denominator = math.pow(1+alpha, days_diff)
    if denominator != 0:
        return 1.0 / denominator
    else:
        return 0.0
    

def get_days_diff(newer_timestamp, older_timestamp):
    sec_diff = newer_timestamp - older_timestamp
    days_diff = sec_diff / 60 / 60 / 24
    return days_diff

def get_confidence_sample_size(sample, max_for_reference=100000):
    #Avoiding overflow for large sample size
   

    return (np.log(1+sample) / float(math.log(1+max_for_reference, 2))).where(sample <= max_for_reference, 1.0)



In [79]:
for i in [0,0.5,1,2,3,4,5,10,20,30,100,200,300,1000,2000,3000,10000,20000,30000, 50000, 90000, 100000, 500000, 900000, 1000000, 2171607]:
    print(i, get_confidence_sample_size(i))



0 0.0
0.5 0.024411410743763327
1 0.041731582304281624
2 0.06614299304804495
3 0.08346316460856325
4 0.09689773339641579
5 0.10787457535232657
10 0.14436755531919657
20 0.183298356035222
30 0.20674645107847822
100 0.2778577004917695
200 0.3192904933647466
300 0.34360197720285013
1000 0.41594812296601125
2000 0.4576496248565576
3000 0.48205100545505175
10000 0.5545232830964639
20000 0.5962518553291584
30000 0.6206622626822822
50000 0.6514162003061013
90000 0.6868039178501281
100000 1.0
500000 1.0
900000 1.0
1000000 1.0
2171607 1.0


In [80]:
timestamp_ref = date_time_to_unix_epoch(datetime.datetime(2016, 6, 29, 3, 59, 59))
decay_factor_default = get_time_decay_factor(publish_time_median_cudf, timestamp_ref, alpha=TIME_DECAY_ALPHA)
print("decay_factor_default", decay_factor_default)


decay_factor_default 0.00020583144472333557


In [195]:
def get_popularity(an_id, a_dict,which_confidence=None):
    try:
        return (float(a_dict.loc[an_id][['ctr']].values[0]), get_confidence_sample_size(a_dict.loc[an_id][['views']].values[0] / float(a_dict.loc[an_id][['distinct_ad_ids']].values[0])) * (1 if not which_confidence else a_dict.loc[an_id][[which_confidence]].values[0]))
    except:
        return (None,None)


In [ ]:
def get_weighted_avg_popularity_from_list(ids_list, confidence_ids_list, pop_dict,which_confidence):
    
    pops = list(filter(lambda x: x[0][0]!=None, [(get_popularity(an_id, pop_dict,which_confidence), confidence) for an_id, confidence in zip(ids_list, confidence_ids_list)]))
    #print("pops",pops)
    
    if len(pops) > 0:
        weighted_avg = sum(map(lambda x: x[0][0]*x[0][1]*x[1], pops)) / float(sum(map(lambda x: x[0][1]*x[1], pops)))
        confidence = max(map(lambda x: x[0][1]*x[1], pops))
        return weighted_avg, confidence
    else:
        return None, None


In [ ]:
def get_weighted_avg_country_popularity_from_list(event_country, ids_list, confidence_ids_list, pop_dict,which_confidence):
    pops = list(filter(lambda x: x[0][0]!=None, [(get_popularity((event_country,an_id), pop_dict,which_confidence), confidence) for an_id, confidence in zip(ids_list, confidence_ids_list)]))
    #print("pops",pops)
    if len(pops) > 0:
        weighted_avg = sum(map(lambda x: x[0][0]*x[0][1]*x[1], pops)) / float(sum(map(lambda x: x[0][1]*x[1], pops)))
        confidence = max(map(lambda x: x[0][1]*x[1], pops))
        return weighted_avg, confidence
    else:
        return None, None


In [ ]:
def get_popularity_score(event_country, ad_id, document_id, source_id, 
      publisher_id, advertiser_id, campaign_id, document_id_event,
      category_ids_by_doc, cat_confidence_level_by_doc, 
      topic_ids_by_doc, top_confidence_level_by_doc,
      entity_ids_by_doc, ent_confidence_level_by_doc,
      output_detailed_list=False):
    probs = []
    
    avg_ctr, confidence = get_popularity(ad_id, ad_id_popularity_cudf,)    
    if avg_ctr != None:
        probs.append(('pop_ad_id', avg_ctr, confidence))
        
    avg_ctr, confidence = get_popularity(document_id, document_id_popularity_cudf)
    if avg_ctr != None:
        probs.append(('pop_document_id', avg_ctr, confidence))  
        
    avg_ctr, confidence = get_popularity((document_id_event, document_id), doc_event_doc_ad_avg_ctr_cudf)
    if avg_ctr != None:
        probs.append(('pop_doc_event_doc_ad', avg_ctr, confidence))
        
        
    if source_id != -1:
        avg_ctr = None
        if event_country != '':
            avg_ctr, confidence = get_popularity((event_country, source_id), source_id_by_country_popularity_cudf)
        if avg_ctr != None:
            probs.append(('pop_source_id_country', avg_ctr, confidence))
            
        avg_ctr, confidence = get_popularity(source_id, source_id_popularity_cudf)        
        if avg_ctr != None:
            probs.append(('pop_source_id', avg_ctr, confidence))
            
            
    if publisher_id != None:
        avg_ctr, confidence = get_popularity(publisher_id, publisher_popularity_cudf)
        if avg_ctr != None:
            probs.append(('pop_publisher_id', avg_ctr, confidence)) 
            
    if advertiser_id != None:
        avg_ctr, confidence = get_popularity(advertiser_id, advertiser_id_popularity_cudf)
        if avg_ctr != None:
            probs.append(('pop_advertiser_id', avg_ctr, confidence)) 
    
    if campaign_id != None:
        avg_ctr, confidence = get_popularity(campaign_id, campaign_id_popularity_cudf)
        if avg_ctr != None:
            probs.append(('pop_campain_id', avg_ctr, confidence))  

    if len(entity_ids_by_doc) > 0: 
        avg_ctr = None
        if event_country != '':
            avg_ctr, confidence = get_weighted_avg_country_popularity_from_list(
              event_country, entity_ids_by_doc, ent_confidence_level_by_doc, 
              entity_id_by_country_popularity_cudf,which_confidence='avg_confidence_level_ent')
        if avg_ctr != None:
            probs.append(('pop_entity_id_country', avg_ctr, confidence))
            
        avg_ctr, confidence = get_weighted_avg_popularity_from_list(
          entity_ids_by_doc, ent_confidence_level_by_doc, 
          entity_id_popularity_cudf,which_confidence='avg_confidence_level_ent') 
        if avg_ctr != None:
            probs.append(('pop_entity_id', avg_ctr, confidence))
            
    
    
    if len(topic_ids_by_doc) > 0:  
        avg_ctr = None
        if event_country != '':
            avg_ctr, confidence = get_weighted_avg_country_popularity_from_list(
              event_country, topic_ids_by_doc, top_confidence_level_by_doc, 
              topic_id_id_by_country_popularity_cudf,which_confidence='avg_confidence_level_top')
        if avg_ctr != None:
            probs.append(('pop_topic_id_country', avg_ctr, confidence))
            
        avg_ctr, confidence = get_weighted_avg_popularity_from_list(
          topic_ids_by_doc, top_confidence_level_by_doc, 
          topic_id_popularity_cudf,which_confidence='avg_confidence_level_top')            
        if avg_ctr != None:
            probs.append(('pop_topic_id', avg_ctr, confidence))
    
    
    if len(category_ids_by_doc) > 0:  
        avg_ctr = None
        if event_country != '':
            avg_ctr, confidence = get_weighted_avg_country_popularity_from_list(
              event_country, category_ids_by_doc, cat_confidence_level_by_doc, 
              category_id_by_country_popularity_cudf,which_confidence='avg_confidence_level_cat')
        if avg_ctr != None:
            probs.append(('pop_category_id_country', avg_ctr, confidence))
        
        avg_ctr, confidence = get_weighted_avg_popularity_from_list(
          category_ids_by_doc, cat_confidence_level_by_doc, 
          category_id_popularity_cudf,which_confidence='avg_confidence_level_cat')
        if avg_ctr != None:
            probs.append(('pop_category_id', avg_ctr, confidence))
    
    #print("[get_popularity_score] probs", probs)
    if output_detailed_list:
        return probs
    
    else:    
        if len(probs) > 0:
            #weighted_avg_probs_by_confidence = sum(map(lambda x: x[1] *  math.log(1+x[2],2), probs)) / float(sum(map(lambda x: math.log(1+x[2],2), probs)))        
            weighted_avg_probs_by_confidence = sum(map(lambda x: x[1] * x[2], probs)) / float(sum(map(lambda x: x[2], probs)))                
            confidence = max(map(lambda x: x[2], probs))
            return weighted_avg_probs_by_confidence, confidence
        else:
            return None, None


In [ ]:
get_popularity(3021, ad_id_popularity_cudf,)  


In [ ]:
get_popularity(12668,document_id_popularity_cudf)

In [ ]:
get_popularity((2861,874764),doc_event_doc_ad_avg_ctr_cudf)

In [ ]:
get_popularity(('AE',6330),source_id_by_country_popularity_cudf)

In [ ]:
get_popularity(6, source_id_popularity_cudf)

In [ ]:
get_popularity(5, publisher_popularity_cudf)

In [ ]:
get_popularity(4, advertiser_id_popularity_cudf)


In [ ]:
get_popularity(30, campaign_id_popularity_cudf)

In [ ]:
get_weighted_avg_popularity_from_list(
            [0,10], [.1,.1], 
              topic_id_popularity_cudf,which_confidence='avg_confidence_level_top')


In [ ]:
get_weighted_avg_country_popularity_from_list(
              'AE', [0,10], [.1,.1], 
              topic_id_by_country_popularity_cudf,which_confidence='avg_confidence_level_top')

In [ ]:
get_weighted_avg_country_popularity_from_list(
              'AE', ['07c145e2973a82ad49230a084b2448a7'], [.1], 
              entity_id_by_country_popularity_cudf,which_confidence='avg_confidence_level_ent')

In [ ]:
get_weighted_avg_popularity_from_list(
          ['07c145e2973a82ad49230a084b2448a7'], [.1], 
          entity_id_popularity_cudf,which_confidence='avg_confidence_level_ent')

In [ ]:
get_weighted_avg_country_popularity_from_list(
              'AE', ['1203'], [.1], 
              category_id_by_country_popularity_cudf,which_confidence='avg_confidence_level_cat')

In [ ]:
get_weighted_avg_popularity_from_list(
              ['1203'], [.1], 
              category_id_popularity_cudf,which_confidence='avg_confidence_level_cat')

In [ ]:
A = train_set_cudf[['document_id_promo','document_id_event']].drop_duplicates().merge(documents_categories_cudf[['document_id_cat','confidence_level_cat','category_id']], left_on='document_id_promo',right_on='document_id_cat',how='inner') \
.merge(documents_categories_cudf[['document_id_cat','confidence_level_cat','category_id']], left_on=['document_id_event'],right_on=['document_id_cat'],how='inner')
A['product'] = A['confidence_level_cat_x'] * A['confidence_level_cat_y']
A.loc[(A['category_id_x']!=A['category_id_y']),'product'] = 0
A = A.groupby(['document_id_promo','document_id_event'])[['product']].agg('sum')
A.reset_index(inplace=True)

In [ ]:
documents_categories_cudf['confidence_squared'] = documents_categories_cudf['confidence_level_cat'] **2
B = documents_categories_cudf.groupby('document_id_cat')[['confidence_squared']].agg('sum')
B['confidence_squared'] = np.sqrt(B['confidence_squared'])
B.reset_index(inplace=True)

In [ ]:
C = A \
.merge(B,left_on='document_id_promo',right_on='document_id_cat') \
.merge(B,left_on='document_id_event',right_on='document_id_cat')
C['final'] = C['product'] / (C['confidence_squared_x'] * C['confidence_squared_y'])

In [198]:
documents_categories_tfidf_cudf = documents_categories_cudf.merge(categories_docs_counts_cudf.to_frame().rename(columns={0:'count'}).reset_index(), left_on='category_id', right_on='category_id')
documents_categories_tfidf_cudf['confidence_level_cat'] = 1.0 * np.log(documents_total/ documents_categories_tfidf_cudf['count'].astype(float)) * documents_categories_tfidf_cudf['confidence_level_cat']

documents_topics_tfidf_cudf = documents_topics_cudf.merge(topics_docs_counts_cudf.to_frame().rename(columns={0:'count'}).reset_index(), left_on='topic_id', right_on='topic_id')
documents_topics_tfidf_cudf['confidence_level_top'] = 1.0 * np.log(documents_total/ documents_topics_tfidf_cudf['count'].astype(float)) * documents_topics_tfidf_cudf['confidence_level_top']

documents_entities_tfidf_cudf = documents_entities_cudf.merge(entities_docs_counts_cudf.to_frame().rename(columns={0:'count'}).reset_index(), left_on='entity_id', right_on='entity_id')
documents_entities_tfidf_cudf['confidence_level_top'] = 1.0 * np.log(documents_total/ documents_entities_tfidf_cudf['count'].astype(float)) * documents_entities_tfidf_cudf['confidence_level_ent']

In [199]:
def cosine_similarity(merged_gdf,x_gdf, y_gdf, index_x, index_y,value_x ,value_y,x_join, merge_x_join, y_join, merge_y_join):
    x_norm = x_gdf
    x_norm['squared_value_x'] = x_norm[value_x]**2

    x_norm= x_norm.groupby([x_join])[['squared_value_x']].agg('sum')
    x_norm['squared_value_x'] = np.sqrt(x_norm['squared_value_x'])
    x_norm.reset_index(inplace=True)
    
    y_norm = y_gdf
    y_norm['squared_value_y'] = y_norm[value_y]**2
    y_norm= y_norm.groupby(y_join)[['squared_value_y']].agg('sum')
    y_norm['squared_value_y'] = np.sqrt(y_norm['squared_value_y'])
    y_norm.reset_index(inplace=True)
    
    
    merged_gdf = merged_gdf[[merge_x_join,merge_y_join]].drop_duplicates().merge(x_gdf,left_on=merge_x_join,right_on=x_join). \
        merge(y_gdf,left_on=merge_y_join,right_on=y_join)
    
    if index_x == index_y:
        index_x += '_x'
        index_y += '_y'
    if value_x == value_y:
        value_x += '_x'
        value_y += '_y'
    merged_gdf['product'] = merged_gdf[value_x] * merged_gdf[value_y]
    merged_gdf['intersections'] = 1
    merged_gdf.loc[merged_gdf[index_x] != merged_gdf[index_y],'product'] = 0
    merged_gdf.loc[merged_gdf[index_x] != merged_gdf[index_y],'intersections'] = 0
    merged_gdf = merged_gdf.groupby([merge_x_join,merge_y_join])[['product','intersections']].agg('sum')
    merged_gdf.reset_index(inplace=True)
    
    
    
    merged_gdf = merged_gdf.merge(x_norm, left_on= merge_x_join, right_on=x_join)\
        .merge(y_norm, left_on=merge_y_join,right_on=y_join)
    
    merged_gdf['cosine_similarity'] = merged_gdf['product'] / (merged_gdf['squared_value_x'] * merged_gdf['squared_value_y'])
    
    return merged_gdf
    

In [200]:

categories_similarity = cosine_similarity(train_set_cudf,documents_categories_tfidf_cudf,documents_categories_tfidf_cudf,'category_id_x','category_id_y','confidence_level_cat','confidence_level_cat','document_id_cat','document_id_promo','document_id_cat','document_id_event')
topics_similarity = cosine_similarity(train_set_cudf,documents_topics_tfidf_cudf,documents_topics_tfidf_cudf,'topic_id_x','topic_id_y','confidence_level_top','confidence_level_top','document_id_top','document_id_promo','document_id_top','document_id_event')
entities_similarity = cosine_similarity(train_set_cudf,documents_entities_tfidf_cudf,documents_entities_tfidf_cudf,'entity_id_x','entity_id_y','confidence_level_ent','confidence_level_ent','document_id_ent','document_id_promo','document_id_ent','document_id_event')

In [201]:
categories_similarity=categories_similarity[['document_id_event','document_id_promo','cosine_similarity']]
topics_similarity=topics_similarity[['document_id_event','document_id_promo','cosine_similarity']]
entities_similarity=entities_similarity[['document_id_event','document_id_promo','cosine_similarity']]


In [ ]:
documents_categories_pd = pd.read_csv(
    DATA_BUCKET_FOLDER + 'documents_categories.csv',
    names=['document_id_cat', 'category_id', 'confidence_level_cat'],
    dtype={'document_id_cat': 'int64','category_id': 'int64','confidence_level_cat': 'float32'},
    skiprows=1)

In [306]:
train_set_pd = train_set_cudf.to_pandas()

In [307]:
documents_categories_grouped = documents_categories_pd.groupby('document_id_cat').agg({'category_id':list,'confidence_level_cat':list})

KeyboardInterrupt: 

In [ ]:
documents_entities_pd = pd.read_csv(
    DATA_BUCKET_FOLDER + 'documents_entities.csv',

    names=['document_id_ent', 'entity_id', 'confidence_level_ent'],
    dtype={'document_id_ent': 'int64','entity_id': 'str','confidence_level_ent': 'float32'},
    skiprows=1)

In [ ]:
entities_categories_grouped = documents_entities_pd.groupby('document_id_ent').agg({'entity_id':list,'confidence_level_ent':list})

In [ ]:
test = train_set_pd[['document_id_promo','document_id_event']].drop_duplicates() \
.merge(documents_categories_grouped,left_on='document_id_promo',right_on='document_id_cat') \
.merge(documents_categories_grouped,left_on='document_id_event',right_on='document_id_cat')


In [ ]:
test.groupby(['document_id_promo','document_id_event']).count()['category_id_x'].max()

In [ ]:
def cosine_sim_simple(cat_id_x,cat_conf_x,cat_id_y,cat_conf_y):
    tot = 0
    for i in range(len(cat_id_x)):
        for j in range(len(cat_id_y)):
            if(cat_id_x[i] == cat_id_y[j]):
                tot += cat_conf_x[i] * cat_conf_y[j]
                
    cat_conf_x = np.array(cat_conf_x)
    cat_conf_y = np.array(cat_conf_y)
    return tot / (np.linalg.norm(cat_conf_x) * np.linalg.norm(cat_conf_y))
        

In [ ]:
def wrapper(test):
    return cosine_sim_simple(test['category_id_x'],test['confidence_level_cat_x'],test['category_id_y'],test['confidence_level_cat_y'])

In [ ]:
def get_promo_event(df,promo,event):
    return df[(df['document_id_promo']==promo) & (df['document_id_event']==event)]

In [ ]:
test['cosine_sim']=test.apply(wrapper,axis=1)

In [ ]:
test

In [ ]:
from math import isclose

In [122]:
for index, row in test.iterrows():
   
    if(not isclose(get_promo_event(categories_similarity,row['document_id_promo'],row['document_id_event'])['cosine_similarity'].iloc[0], row['cosine_sim'],abs_tol=1e-6)):
        print(row)
        

In [82]:
#Adding ad_views and popularity

In [238]:
FV = train_set_cudf.merge(ad_id_popularity_cudf[['ad_id','views','ctr','confidence']].rename(columns={'views':'ad_views','ctr':'pop_ad_id','confidence':'pop_ad_id_conf'}), left_on='ad_id',right_on='ad_id',how='left')

In [211]:
#Adding doc_views and popularity

In [239]:
FV = FV.merge(document_id_popularity_cudf[['document_id_promo','views','ctr','confidence']].rename({'views':'doc_views','ctr':'pop_document_id','confidence':'pop_document_id_conf'}), left_on='document_id_promo',right_on='document_id_promo',how='left')

In [240]:
#Note here that user_views comes from user profiles, so is not a viable option immediately

In [241]:
#Add doc_ad_days_since_published


In [242]:
TIMESTAMP_DELTA = 1465876799998
FV['dt_timestamp_event'] = cudf.to_datetime(FV['timestamp_event'].astype(int)+TIMESTAMP_DELTA,unit='ms')
FV['doc_ad_days_since_published'] = FV['dt_timestamp_event'].astype(int) //  (1000000000 * 86400) - FV['publish_time'].astype(int) // (1000 * 86400)

In [243]:
# Add doc_event_days_since_published

In [244]:
FV['doc_event_days_since_published'] = FV['dt_timestamp_event'].astype(int) //  (1000000000 * 86400) - FV['publish_time_doc_event'].astype(int) // (1000 * 86400)

In [245]:
#Check if weekend

In [246]:
FV['event_weekend'] = FV['dt_timestamp_event'].dt.weekday > 5

In [247]:



FV['tz_area'] = FV['event_country']
FV['tz_area'] = FV['tz_area'].where(~FV['event_country'].isin(['CA','US']),other=FV['event_country_state'])

FV = FV.merge(geo_code,left_on='tz_area',right_on='geo_code',how='left')



In [248]:
FV['event_hour'] = ((FV['dt_timestamp_event'].dt.hour + FV['utc_dst_time_offset_cleaned'])%24).applymap(get_hour_bin)

In [249]:
# Changing categorical countries to less value if below threshold (CATEGORIFY)

In [250]:
FV['event_country'].loc[~FV['event_country'].isin(event_country_values_counts['event_country'])] = 'less'

In [251]:
# Changing categorical states to less value if below threshold (CATEGORIFY)

In [252]:
FV['event_country_state'].loc[~FV['event_country_state'].isin(event_country_state_values_counts['event_country_state'])] = 'less'

In [253]:
# Changing categorical geo_location to less value if below threshold (CATEGORIFY)

In [254]:
FV['geo_location_event'].loc[~FV['geo_location_event'].isin(event_geo_location_values_counts['geo_location_event'])] = 'less'
FV.rename(columns={'geo_location_event':'event_geo_location'},inplace=True)

In [255]:
#Readjusting traffic source per source

In [256]:
FV['traffic_source'] = FV['traffic_source_pv'] -1

In [257]:
#Readjusting event platform

In [258]:
FV['event_platform'] = FV['platform_event'] -1

In [259]:
# column renaming

In [260]:
FV = FV.merge(source_id_popularity_cudf[['source_id','ctr','confidence']].rename({'ctr':'pop_source_id','confidence':'pop_source_id_conf'}), left_on='source_id',right_on='source_id',how='left')

In [261]:
FV = FV.merge(publisher_popularity_cudf[['publisher_id','ctr','confidence']].rename({'ctr':'pop_publisher_id','confidence':'pop_publisher_id_conf'}), left_on='publisher_id',right_on='publisher_id',how='left')

In [262]:
FV = FV.merge(advertiser_id_popularity_cudf[['advertiser_id','ctr','confidence']].rename({'ctr':'pop_advertiser_id','confidence':'pop_advertiser_id_conf'}), left_on='advertiser_id',right_on='advertiser_id',how='left')

In [263]:
FV = FV.merge(campaign_id_popularity_cudf[['campaign_id','ctr','confidence']].rename({'ctr':'pop_campaign_id','confidence':'pop_campaign_id_conf'}), left_on='campaign_id',right_on='campaign_id',how='left')

In [264]:
FV = FV.merge(categories_similarity.rename(columns={'cosine_similarity':'doc_event_doc_ad_sim_categories'}), left_on=['document_id_event','document_id_promo'],right_on=['document_id_event','document_id_promo'],how='left')

In [265]:
FV = FV.merge(entities_similarity.rename(columns={'cosine_similarity':'doc_event_doc_ad_sim_entities'}), left_on=['document_id_event','document_id_promo'],right_on=['document_id_event','document_id_promo'],how='left')

In [266]:
FV = FV.merge(topics_similarity.rename(columns={'cosine_similarity':'doc_event_doc_ad_sim_entities'}), left_on=['document_id_event','document_id_promo'],right_on=['document_id_event','document_id_promo'],how='left')

In [268]:
FV.rename(columns={'source_id':'doc_ad_source_id',
                   'publisher_id':'doc_ad_publisher_id',
                   'source_id_doc_event':'doc_event_source_id',
                   'publisher_doc_event':'doc_event_publisher_id',
                   'document_id_event':'doc_event_id',
                   'document_id_doc':'doc_id',
                   'advertiser_id':'ad_advertiser',
                   
                   },inplace=True)

In [269]:
FV

,display_id,ad_id,clicked,document_id_promo,campaign_id,ad_advertiser,doc_id,doc_ad_source_id,publish_time,doc_ad_publisher_id,...,pop_source_id_conf,pop_publisher_id,pop_publisher_id_conf,pop_advertiser_id,pop_advertiser_id_conf,pop_campaign_id,pop_campaign_id_conf,doc_event_doc_ad_sim_categories,doc_event_doc_ad_sim_entities_x,doc_event_doc_ad_sim_entities_y
0,3773,51955,1,971259,6797,551,971259,2566,2015-11-30,415,...,0.095913418,0.370860927,0.091815307,0.372413793,0.095913418,0.5,0.170576292,0.98677324,0.0,0.0
1,3773,95103,0,1098103,10945,1919,1098103,6746,2016-01-24,2581,...,0.213806855,0.025316456,0.213806855,0.02540913,0.214305533,0.023746702,0.214829958,0.0,0.0,0.0
2,3773,140940,0,1183136,18034,102,1183136,3199,2015-10-01,341,...,0.133056633,0.105357143,0.133056633,0.105357143,0.133056633,0.140186916,0.176742284,0.974852199,0.0,0.0
3,3773,141024,0,1323334,12689,2812,1323334,6604,NaT,225,...,0.136884094,0.355828221,0.136039413,0.377952756,0.135329534,0.318181818,0.159740945,0.0,0.0,0.0
4,3773,235535,0,1334819,18461,2889,1334819,4679,NaT,1825,...,0.170576292,0.0,0.170576292,0.0,0.170576292,0.0,0.170576292,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,19013,289104,1,1687165,28025,3899,1687165,13568,2016-05-11,5674,...,0.161466625,0.239336493,0.161466625,0.239336493,0.161466625,0.256329114,0.19453194,0.012171082,null,0.0
99995,19007,280303,0,1663937,27307,3391,1663937,13555,2015-09-15,5665,...,0.087434227,0.097222222,0.087434227,0.097222222,0.087434227,null,null,0.0,null,0.0
99996,19007,290544,1,834781,3971,1724,834781,6587,NaT,2506,...,0.06122549,0.166666667,0.06122549,0.193548387,0.060293158,null,null,0.084039883,null,0.994783832
99997,14308,23288,0,819061,3641,629,819061,28,2015-07-06,1270,...,0.163040726,0.333333333,0.163040726,0.37704918,0.118009362,0.230769231,0.198428979,0.0,null,0.0


In [280]:
document_id_popularity_cudf[document_id_popularity_cudf['document_id_promo']==774527]

,document_id_promo,ctr,views,distinct_ad_ids,confidence


In [278]:
documents_meta_cudf


,document_id_doc,source_id,publish_time,publisher_id
0,269941,1,2014-03-02 00:00:00,603
1,269947,1,2014-03-01 17:00:00,603
2,269948,1,2014-03-10 00:00:00,603
3,343844,1,2014-06-17 00:00:00,603
4,377379,1,2015-09-10 00:00:00,603
...,...,...,...,...
2996811,1513582,14053,2016-05-20 16:00:00,6066
2996812,1213335,14054,2016-03-05 12:00:00,6067
2996813,1088537,14055,2015-07-01 17:00:00,6068
2996814,1153949,14056,2016-03-12 13:00:00,6069


774527